In [1]:
from torch.utils.data import DataLoader
from torch import optim
import time
import csv

from lib.aperture_dataset import ApertureDataset
from lib.lenet import LeNet
from lib.fit import fit

log_fname = 'results_%s.csv' % time.strftime('%Y%m%d-%H%M%S')

k = 4
fname_train = 'training_data/training_data.h5'
fname_validate = 'training_data/validation_data.h5'
save_path = 'save_here'

# model_params_dict = {
#     'input_dim': 130, # TODO: move?
#     'output_dim': 130,
#     'layer_width': 260,
#     'num_hidden_layers': 4,
# }

model_params_dict = {
    'input_dim': 65, # TODO: move?
    'output_dim': 130,
    'layer_width': 260,
    'num_hidden_layers': 4,
}

input_dim = model_params_dict['input_dim']
output_dim = model_params_dict['output_dim']

num_samples = 10 ** 5
dat_train = ApertureDataset(fname_train, num_samples, k)


num_samples = 10 ** 4
dat_train2 = ApertureDataset(fname_train, num_samples, k)

num_samples = 10 ** 4
dat_validate = ApertureDataset(fname_validate, num_samples, k)


/Users/zhanwenchen/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
lr = 0.5
momentum = 0
# TODO: switch up learning rates

using_cuda = False

try_batch_sizes = [32, 64, 128]
try_hidden_sizes = range(10, 720, 10)
try_num_hidden_layers = range(1, 7)
try_kernel_sizes = range(2, 10)
try_num_kernels = [1]

run_index = 0
with open(log_fname, 'a') as f:
    writer = csv.writer(f)
    header = 'index, cost, diff, kernel_size, num_kernels, num_hidden_layers, hidden_size, batch_size'
    
    print("| run | cost       | diff   | kernel_size | num_kernels | num_hidden_layers | hidden_size | batch_size |")
    print("| --- | ---------- | ------ | ----------- | ----------- | ----------------- | ----------- | ---------- |")
    writer.writerow(header)
    
    for batch_size in try_batch_sizes:
        
        train_loader = DataLoader(dat_train, batch_size=batch_size, shuffle=True, num_workers=1)
        train_loader2 = DataLoader(dat_train2, batch_size=batch_size, shuffle=False, num_workers=1)
        validate_loader = DataLoader(dat_validate, batch_size=batch_size, shuffle=False, num_workers=1)        
    
        for kernel_size in try_kernel_sizes:
            # TODO: in lenet.py, change fc layer input_size to allow any num_kernels
            for num_kernels in try_num_kernels:

                for num_hidden_layers in try_num_hidden_layers:

                    for hidden_size in try_hidden_sizes:
                    
                        model = LeNet(input_dim, output_dim, hidden_size, num_hidden_layers, batch_size)
                        optimizer = optim.SGD(model.parameters(), lr, momentum)
                        if using_cuda: model.cuda()
                        cost, diff = fit(model, train_loader, train_loader2, validate_loader, optimizer, save_path, cuda=using_cuda)
        
                        row = '%03d, %0.5f, %2.3f%%, %02d, %02d, %d, %03d, %03d' % (run_index, cost, diff, kernel_size, num_kernels, num_hidden_layers, hidden_size, batch_size)
                        
                        print("| %03d | %0.5f    | %2.3f%% | %02d          | %02d          | %d                 | %03d         | %03d        |" % (run_index, cost, diff, kernel_size, num_kernels, num_hidden_layers, hidden_size, batch_size))
                        writer.writerow(row)
                    
                        run_index += 1
    

print("\nIt's all over.")

| run | cost       | diff   | kernel_size | num_kernels | num_hidden_layers | hidden_size | batch_size |
| --- | ---------- | ------ | ----------- | ----------- | ----------------- | ----------- | ---------- |


ZeroDivisionError: float division by zero